In [1]:

from mindquantum.algorithm.nisq import HardwareEfficientAnsatz,RYFull
from mindquantum.core.parameterresolver import  PRGenerator
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, I,CNOT
from mindquantum.core.circuit import Circuit,UN
import mindspore as ms
import pickle
from mindquantum.core.parameterresolver import PRGenerator
import random
from mindspore import Tensor,ops
import tensorcircuit as tc
import tensorflow as tf
import mindspore.numpy as mnp
from DQAS_tool import wash_pr,Mindspore_ansatz_micro
from DQAS_tool import  sampling_from_structure,zeroslike_grad_nnp_micro,nmf_gradient,vag_nnp_micro,DQAS_accuracy
import sys
from typing import Union
sys.path.append('..')
from Test_tool import Test_ansatz
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.core.circuit import change_param_name,apply

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR
Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [2]:
pr_pool = PRGenerator('pool')
parameterized_circuit= \
[UN(RX(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RX(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RX(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RX(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),
 UN(RY(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RY(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RY(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RY(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),
 UN(RZ(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RZ(pr_pool.new()),maps_obj=[1])+I.on(0),
 UN(RZ(pr_pool.new()),maps_obj=[0],maps_ctrl=[1]),
 UN(RZ(pr_pool.new()),maps_obj=[1],maps_ctrl=[0]),]


unparameterized_circuit = \
[UN(X,maps_obj=[0],maps_ctrl=[1]),
 UN(X,maps_obj=[1],maps_ctrl=[0]),
 UN(Z,maps_obj=[0],maps_ctrl=[1]),
 UN(Z,maps_obj=[1],maps_ctrl=[0]),
 UN(Y,maps_obj=[0],maps_ctrl=[1]),
 UN(Y,maps_obj=[1],maps_ctrl=[0]),
 ]
ansatz_pr = PRGenerator('ansatz')
shape_parametized = len(parameterized_circuit)
shape_unparameterized = len(unparameterized_circuit)
num_layer=7
shape_nnp = (7,num_layer,shape_parametized)
shape_stp = (num_layer,shape_unparameterized+shape_parametized)
stddev = 0.03
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)
ops_onehot = ops.OneHot(axis=-1)

In [3]:
with open('training_history-2.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
data['ansatz_params_history'][-1].shape

(7, 7, 12)

In [14]:
nnp = data['ansatz_params_history'][-1]
stp = data['structure_distribution_history'][-1]
candidate = data['best_candidates_history'][-1]
K = tc.set_backend("tensorflow")
ops_onehot = ops.OneHot(axis=-1)
best_structure = ops_onehot(ms.Tensor(candidate),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(1e-3))
epochs = 600
for epoch in range(epochs):

    infd, gnnp = vag_nnp_micro(Structure_params=best_structure,Ansatz_params=nnp,paramerterized_pool=parameterized_circuit,unparamerterized_pool=unparameterized_circuit,num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
    grad_nnp_zeroslike = zeroslike_grad_nnp_micro(batch_sturcture=best_structure,grad_nnp=gnnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)

    print(infd,gnnp)
    nnp_tf = network_opt.update(tf.convert_to_tensor(grad_nnp_zeroslike), tf.convert_to_tensor(nnp))
    nnp = nnp_tf.numpy()
    if epoch % 10 == 0 or epoch == epochs - 1:
        print(epoch, "loss: ", infd)
        
        
        
        
        

0.48445338 (Tensor(shape=[49], dtype=Float32, value= [-2.28515361e-04,  1.79444069e-05, -1.70418294e-04,  1.23580408e-04,  6.85437990e-04,  4.90936036e-05, -2.25763913e-04, -1.24121550e-04,  4.49725048e-05, -3.53595933e-05,  3.80737220e-05, -3.86069514e-05, 
  5.83371002e-05, -8.41751626e-06, -1.52998909e-05,  1.29855267e-04,  3.44360524e-05, -5.44826726e-05, -6.11936703e-05,  1.39514625e-04,  1.25798862e-04,  5.98455335e-05,  6.64877298e-05,  2.59777898e-05, 
 -2.16955759e-05, -1.41534510e-05,  2.84227426e-04, -1.06255618e-04,  4.90936036e-05,  1.30451908e-05,  1.10006408e-06,  1.88655540e-04,  5.29951330e-06, -1.22019257e-04, -2.73132493e-04,  1.39514625e-04, 
  3.19447572e-05, -2.40883164e-04,  9.05149409e-06, -1.33617839e-04, -5.61986060e-04,  8.23490336e-05, -1.22019257e-04,  5.94478624e-05,  2.73008103e-04, -2.50365938e-05, -2.53212929e-04,  8.18594417e-05, 
  2.35940257e-04]),)
0 loss:  0.48445338
0.48444808 (Tensor(shape=[49], dtype=Float32, value= [-2.63232767e-04, -4.48629771

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107f0f6a0>>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


0.4840263 (Tensor(shape=[49], dtype=Float32, value= [-3.93916998e-05,  1.04497099e-06, -1.48292194e-04, -6.73184404e-05,  4.49716317e-04,  3.41301615e-07, -2.46627023e-04, -7.09707892e-05,  1.45506874e-06, -5.83545880e-05,  4.93389791e-07, -1.40513685e-05, 
  3.31739866e-05,  1.63041034e-06,  5.22662873e-08,  9.45874672e-06,  1.79978360e-06,  7.19937361e-06,  3.34839797e-06,  5.06465540e-06,  4.21793601e-07,  2.49457826e-05,  1.76810267e-04, -1.29051693e-06, 
 -7.76610761e-07, -1.13060548e-06,  2.91134289e-04, -1.39474039e-04,  3.41301615e-07, -1.55948710e-05, -4.69598763e-06, -3.21469561e-05, -2.55097211e-06, -1.73542248e-05, -2.77319195e-05,  5.06465540e-06, 
 -1.68738541e-06, -2.73144455e-04,  3.06343391e-05, -1.08917156e-04, -3.69205372e-04,  2.00244831e-05, -1.73542248e-05,  6.04925299e-05, -2.10221806e-05,  2.17405672e-04,  1.32151090e-05,  1.08372275e-04, 
 -2.43831619e-05]),)
140 loss:  0.4840263
0.48402336 (Tensor(shape=[49], dtype=Float32, value= [-3.90000350e-05,  1.03282548

In [9]:
ansatz = Mindspore_ansatz_micro(Structure_p=best_structure,parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,num_layer=num_layer,n_qbits=8)

In [13]:
len(ansatz.ansatz_params_name)

49

In [13]:
def best_from_structure(structures: np.array)->Tensor:
    prob = ops.Softmax()(ms.Tensor(structures))
    return ops.Argmax(axis=-1)(prob)

def best_from_structure(structures: np.array)->Tensor:
    return ops.Argmax(axis=-1)(ms.Tensor(structures))

In [5]:
from DQAS_tool import zeroslike_grad_nnp_micro
grad_nnp_zeroslike = zeroslike_grad_nnp_micro(batch_sturcture=stp_onehot,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)

In [ ]:
from DQAS_tool import  sampling_from_structure,zeroslike_grad_nnp_micro,nmf_gradient,vag_nnp_micro,DQAS_accuracy
#from DQAS_tool import  DQASAnsatz_from_result,DQAS_accuracy
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.1))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = False
# 设置超参数
epochs = 800
batch_size=100
avcost1 = 0
ops_onehot = ops.OneHot(axis=-1)
batch_loss_history=[] # 记录每个epoch的batch_size损失值
structure_distribution_history=[] # 记录每个epoch的结构参数
ansatz_params_history=[] # 记录每个epoch的网络参数
best_candidates_history=[] # 记录每个epoch的最佳候选
acc_history = [] #记录每个epoch的准确率

 
for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
    #print(batch_structure.shape)
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:
        #print(ops.Argmax()(i))          
        infd, grad_nnp = vag_nnp_micro(Structure_params=i,
                                    Ansatz_params=nnp,
                                    paramerterized_pool=parameterized_circuit,  unparamerterized_pool=unparameterized_circuit,
                                    num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        
        grad_nnp_zeroslike = zeroslike_grad_nnp_micro(batch_sturcture=i,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)
        gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=stp.shape[1])
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(ms.Tensor(grad_nnp_zeroslike,dtype=ms.float64))
        grad_stps.append(gs)

      
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    # print(f'loss={infd}\ngrad_nnp={gnnp}\ngrandient_stp={gstp_averge}')
    
    gnnp_tf = tf.convert_to_tensor(gnnp.asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
     # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()

    batch_loss_history.append(avcost1)
    structure_distribution_history.append(stp)
    ansatz_params_history.append(nnp)
    #best_candidates_history.append(best_from_structure(cand_preset.asnumpy()))
    cand_preset = best_from_structure(stp)
    best_candidates_history.append(cand_preset.asnumpy())
    

    if epoch % 1 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched平均损失: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        print("最好的候选结构:",cand_preset)
        stp_for_test = ops_onehot(ms.Tensor(cand_preset),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
        test_ansatz = Mindspore_ansatz_micro(Structure_p=stp_for_test,
                                            parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,
                                            num_layer=num_layer,
                                            n_qbits=8)
        
        ansatz_parameters=[]
        for each_sub in range(7):
            for layerIndex,i in enumerate(cand_preset):
                if i >=len(parameterized_circuit):
                    continue
                else:
                    ansatz_parameters.append(nnp[each_sub,layerIndex,i])
        
        acc = DQAS_accuracy(ansatz=test_ansatz,Network_params=ansatz_parameters,n_qbits=8)
        acc_history.append(acc)
        print(f'二分类准确率 Acc ={acc*100}% ')
        
        #我想每一轮结束 保存batch_loss_history、structure_distribution_history、ansatz_params_history、best_candidates_history、acc_history
                # 保存数据
        with open('training_history-2.pkl', 'wb') as f:
            pickle.dump({
                'batch_loss_history': batch_loss_history,
                'structure_distribution_history': structure_distribution_history,
                'ansatz_params_history': ansatz_params_history,
                'best_candidates_history': best_candidates_history,
                'acc_history': acc_history
            }, f)
        
        

In [ ]:
batch_loss_history

In [ ]:
from DQAS_tool import Test_ansatz
value = Test_ansatz(ansatz=ansatz,learning_rate=0.1,epochs=20)

In [ ]:
max(value)


In [1]:
import pickle

In [2]:
with open('training_history.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data['ansatz_params_history'][-1]